In [1]:
from langchain.schema import Document

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
from langchain.vectorstores import FAISS

In [4]:
from langchain.chains import RetrievalQA

In [5]:
from langchain.prompts import PromptTemplate

In [40]:
from typing import Optional, List, Dict, Mapping, Any

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
from sentence_transformers import SentenceTransformer

In [9]:
import json

In [10]:
import pandas as pd

In [11]:
from io import StringIO

In [45]:
import requests

In [38]:
from langchain.llms.base import LLM

In [12]:
def inspect_pages_data(pages_data, max_pages=2, max_chunks=3, max_text_len=1000, max_rows=50, max_cols=10):
    print(f"Anzahl Seiten: {len(pages_data)}")
    if len(pages_data) == 0:
        print("Keine Seiten im Datenobjekt.")
        return

    for i, page in enumerate(pages_data[:max_pages]):
        print(f"\n=== Seite {i+1} ===")
        print(f"Typ: {type(page)}")
        print(f"Keys: {list(page.keys())}")
        print(f"Seiten-Nummer: {page.get('page_number')}")
        print(f"Dateiname: {page.get('file_name')}")
        print(f"Text (erste {max_text_len} Zeichen): {page.get('text', '')[:max_text_len]!r}")

        chunks = page.get("chunks", [])
        print(f"Anzahl Chunks: {len(chunks)}")
        for j, chunk in enumerate(chunks[:max_chunks]):
            if isinstance(chunk, tuple) and len(chunk) == 2:
                bbox, text = chunk
                print(f"  Chunk {j+1}: bbox={bbox}, text={text[:max_text_len]!r}")
            else:
                print(f"  Chunk {j+1}: {str(chunk)[:max_text_len]!r}")

        tables = page.get("tables", [])
        print(f"Anzahl Tabellen: {len(tables)}")
        for k, table in enumerate(tables):
            print(f"\n  Tabelle {k+1}: Typ={type(table)} Größe={table.shape if hasattr(table, 'shape') else 'unbekannt'}")
            if hasattr(table, "head"):
                # Ausgabe der ersten max_rows Zeilen, max_cols Spalten, als Text
                print(table.iloc[:max_rows, :max_cols].to_string(index=False))
            else:
                print(str(table)[:max_text_len])

In [ ]:
# Nur mit transformers
def chunk_document(pages: List[Dict[str, Any]], max_tokens: int = 512) -> List[Dict[str, Any]]:
    """Fügt Text und Tabellen aus Seiten zusammen und chunked sie."""
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # anpassbar

    chunks = []
    current_chunk = ""
    current_meta = []

    for page in pages:
        full_text = page["text"].strip()

        # Optional: Tabellen als Text anhängen
        for df in page["tables"]:
            full_text += "\n\n" + df.to_string(index=False)

        # Chunking per Token-Limit
        tokens = tokenizer.tokenize(current_chunk + full_text)
        if len(tokens) > max_tokens:
            chunks.append({
                "text": current_chunk.strip(),
                "meta": current_meta
            })
            current_chunk = full_text
            current_meta = [page]
        else:
            current_chunk += "\n\n" + full_text
            current_meta.append(page)

    if current_chunk:
        chunks.append({
            "text": current_chunk.strip(),
            "meta": current_meta
        })

    return chunks

In [13]:
def build_documents_from_pages(pages_data):
    """
    Wandelt die Liste von Seiten-Dictionaries (aus DeepDoctection) 
    in eine Liste von LangChain Document-Objekten um.
    """
    documents = []
    for page in pages_data:
        # Text der Seite
        page_text = page["text"].strip()

        # Tabellen als Text anfügen (optional)
        for df in page.get("tables", []):
            page_text += "\n\n" + df.to_string(index=False)

        # Metadaten mitgeben
        metadata = {
            "page_number": page["page_number"],
            "file_name": page.get("file_name", None),
        }

        doc = Document(page_content=page_text, metadata=metadata)
        documents.append(doc)

    return documents

In [14]:
def import_pages_data_from_json(input_file):
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    pages_data = []
    for page_entry in data:
        page = {
            "page_number": page_entry.get("page_number"),
            "file_name": page_entry.get("file_name"),
            "document_id": page_entry.get("document_id"),
            "image_id": page_entry.get("image_id"),
            "width": page_entry.get("width"),
            "height": page_entry.get("height"),
            "text": page_entry.get("text"),
            "tables": []
        }

        for csv_str in page_entry.get("tables", []):
            # CSV-String zurück in DataFrame konvertieren
            df = pd.read_csv(StringIO(csv_str))
            page["tables"].append(df)

        pages_data.append(page)

    return pages_data

In [67]:
def load_vectorstore(speicherpfad):
    # Gleicher Embedding-Model-Name wie beim Speichern
    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    # Vectorstore laden
    vectorstore = FAISS.load_local(speicherpfad, embeddings=embedding_model, allow_dangerous_deserialization=True)
    return vectorstore

In [41]:
class LocalLlamaLLM(LLM):
    endpoint: Optional[str] = None  # Optional mit Default None

    def __init__(self, device: str = "cpu", **kwargs):
        super().__init__(**kwargs)
        if device == "gpu":
            self.endpoint = "http://llm-gpu:5001/completion"
        else:
            self.endpoint = "http://llm-cpu:5000/completion"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = requests.post(self.endpoint, json={"prompt": prompt, "n_predict": 100})
        response.raise_for_status()
        data = response.json()
        return data.get("content", "")

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"endpoint": self.endpoint}

    @property
    def _llm_type(self) -> str:
        return "local_llama"

In [57]:
def print_clean_result (antwort):
    clean_result = antwort["result"].replace("Answer:", "").strip()
    print(clean_result)
    return  

In [15]:
pages_data = import_pages_data_from_json(input_file="/notebooks/json/extracted_pages_data.json")
print(f"{len(pages_data)} Seiten wurden geladen.")

2 Seiten wurden geladen.


In [16]:
inspect_pages_data(pages_data)

Anzahl Seiten: 2

=== Seite 1 ===
Typ: <class 'dict'>
Keys: ['page_number', 'file_name', 'document_id', 'image_id', 'width', 'height', 'text', 'tables']
Seiten-Nummer: 0
Dateiname: 2024-nachhaltigkeitsbericht_tab_0.pdf
Text (erste 1000 Zeichen): 'HENKEL NACHHALTIGKEITSBERICHT 2024\n( QB\n161\nVORWORT\nREFERENZ- UND\nBERICHTSRAHMEN\nALLGEMEINE ANGABEN\n(ESRS2)\nKLIMAWANDEL (ESRS E1)\nUMWELTVERSCHMUTZUNG\n(ESRS E2)\nWASSER- UND MEERES-\nRESSOURCEN (ESRS E3)\nBIOLOGISCHE VIELFALTUND\nOKOSYSTEME (ESRS E4)\nRESSOURCENNUTZUNG\nUND KRESLAUPWIRISCHAFT\n(ESRS E5)\nARBEITSKRAFTE DES UNTER-\nNEHMENS (ESRS S1)\nARBEITSKRAFTE IN DERWERT-\nSCHOPFUNGSKETTE (ESRS S2)\nBETROFFENE GEMEINSCHAFTEN\n(ESRS S3)\nVERBRAUCHER:INNEN UND\nENDNUTZERINNEN (ESRS S4)\nUNTERNEHMENSFOHRUNG\n(ESRS G1)\nWEITERE INFORMATIONEN\nTHG-Brutoemisionen der Kategorien Scope 1,2und3 3s sowie THG-Gesamtemissionen (MDR-T_80d,80e, 80j, E1-4340,346.E1-6AR41AR4REHG,44EM632aS26,E1-6.51,E1-6.53)\nTHG-Emissionen\nFORTSETZUNG DER TABELLE 

In [22]:
# Beispiel: Wandele pages_data um in LangChain documents
documents = build_documents_from_pages(pages_data)

In [24]:
# Jetzt Text in Chunks splitten
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
alle_chunks = text_splitter.split_documents(documents)
    
print(f"Aus {len(documents)} Seiten wurden {len(alle_chunks)} Chunks erzeugt.")

Aus 2 Seiten wurden 14 Chunks erzeugt.


In [32]:
for i, chunk in enumerate(alle_chunks):
    print(f"\n=== Chunk {i+1} ===")
    print(chunk.page_content[:500] + "...")
    print("Seite:", chunk.metadata.get("page_number"))


=== Chunk 1 ===
HENKEL NACHHALTIGKEITSBERICHT 2024
( QB
161
VORWORT
REFERENZ- UND
BERICHTSRAHMEN
ALLGEMEINE ANGABEN
(ESRS2)
KLIMAWANDEL (ESRS E1)
UMWELTVERSCHMUTZUNG
(ESRS E2)
WASSER- UND MEERES-
RESSOURCEN (ESRS E3)
BIOLOGISCHE VIELFALTUND
OKOSYSTEME (ESRS E4)
RESSOURCENNUTZUNG
UND KRESLAUPWIRISCHAFT
(ESRS E5)
ARBEITSKRAFTE DES UNTER-
NEHMENS (ESRS S1)
ARBEITSKRAFTE IN DERWERT-
SCHOPFUNGSKETTE (ESRS S2)
BETROFFENE GEMEINSCHAFTEN
(ESRS S3)
VERBRAUCHER:INNEN UND
ENDNUTZERINNEN (ESRS S4)
UNTERNEHMENSFOHRUNG
(ESRS...
Seite: 0

=== Chunk 2 ===
0                                                                                  1          2        3                   4                                    5                                       6    7    8    9
NaN                                                                                NaN        NaN      NaN                 NaN                                  NaN                                     NaN  NaN  NaN  NaN
NaN              

In [33]:
# Embeddings mit HuggingFace Sentence Transformers (MiniLM)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [34]:
# VectorStore erzeugen
vectorstore = FAISS.from_documents(alle_chunks, embeddings)

In [36]:
# VectorStore speichern
speicherpfad = "/notebooks/vectorstore/"
vectorstore.save_local(speicherpfad)
print(f"Vektordatenbank gespeichert unter: {speicherpfad}")

Vektordatenbank gespeichert unter: /notebooks/vectorstore/


In [68]:
# Vectorstore laden
# speicherpfad = "/notebooks/vectorstore/"
# vectorstore = load_vectorstore(speicherpfad)

In [42]:
# Lokales LLM initialisieren
llm = LocalLlamaLLM(device="gpu")

In [43]:
# RetrievalQA Chain bauen (gemeinsam für alle PDFs)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [46]:
# Frage
frage = "Extrahiere aus dem Dokument alle Informationen zu THG-Emissionen!"
antwort = qa_chain.invoke({"query": frage})
print("Antwort:", antwort)

Antwort: {'query': 'Extrahiere aus dem Dokument alle Informationen zu THG-Emissionen!', 'result': ' \n\n* THG-Emissionen insgesamt: 127471255 tCO2e\n* THG-Emissionen insgesamt (marktbezogen): 127471255 tCO2e (tCO2e)\n* THG-Emissionen insgesamt (SBTI-Klimaziel Geltungsbereich): 12747125'}


In [50]:
# Frage
frage = "Wie hoch sind die Scope-1-THG-Bruottomissionen im Jahr 2024!"
antwort = qa_chain.invoke({"query": frage})
print("Antwort:", antwort)

Antwort: {'query': 'Wie hoch sind die Scope-1-THG-Bruottomissionen im Jahr 2024!', 'result': '\nAnswer: 17.990.115'}


In [47]:
# Dein eigener Prompt mit der Einschränkung
custom_prompt_template = """Beantworte die folgende Frage basierend auf dem folgenden Kontext.
Suche alle Zahlen in dem Text, die einen Nachhaltigkeitsbezug haben

Kontext: {context}

Frage: {question}
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=custom_prompt_template
)

# 2. RetrievalQA mit eigenem Prompt erstellen
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

In [52]:
# Frage
frage = "Gebe für jede im Dokument gefundene Zahl zu Emissionen mit einem von Dir erkannten Bezug folgendes aus: {Bezug, Zahl} Keine weitere Ausgabe!"
antwort = qa_chain.invoke({"query": frage})
print("Antwort:", antwort)

Antwort: {'query': 'Gebe für jede im Dokument gefundene Zahl zu Emissionen mit einem von Dir erkannten Bezug folgendes aus: {Bezug, Zahl} Keine weitere Ausgabe!', 'result': '\n\n\n\nAnswer:\n{SBTi-Klimaziel Geltungsbereich, 18768446}\n{Scope-i-THG-emissionen, NaN}\n{Biogen, NaN}'}


In [59]:
print_clean_result (antwort)

{SBTi-Klimaziel Geltungsbereich, 18768446}
{Scope-i-THG-emissionen, NaN}
{Biogen, NaN}
